In [2]:
from IPython.display import display, HTML

# Background

"LOINC is a common language (set of identifiers, names, and codes) for identifying health measurements, observations, and documents. If you think of an observation as a "question" and the observation result value as an "answer." (https://loinc.org/)

We have a version of loinc included in this project, however it is not guaranteed to be the most up to date set of codes. 

To download LOINC and set it up to run with this notebook:

**You will need to make an account at https://loinc.org/ to download the files.**

1. Go to https://loinc.org/downloads/loinc/ 
2. Download "LOINC Table File (CSV)" 
3. Go to https://loinc.org/downloads/accessory-files/
4. Download "LOINC Panels and Forms File"
5. Unzip both of these into the same directory (./loinc is default for this code)
6. Update directory path below to point to their location

The code below will set up LOINC as a psql database. Enjoy!

In [2]:
loinc_directory = 'loinc'
psql_username = 'postgres'
psql_password = '123'
loinc_dbname = 'loinc'

In [3]:
import loinc
dbm = loinc.LOINC_db_manager(psql_username, psql_password, loinc_dbname)

In [5]:
dbm.db_create()
dbm.add_loinc_tables(loinc_directory)
dbm.add_panels(loinc_directory)
dbm.add_indicies()

C:\Users\genkinjz\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2825: DtypeWarning: Columns (8,15,17,18,25,26,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


You should now have a PSQL database set up for LOINC on your system. Lets test it out!

In [6]:
import pandas as pd
from utils import psql_connect

In [7]:
conn = psql_connect(psql_username, psql_password, loinc_dbname)
df_loincs = pd.read_sql_query('SELECT * FROM loinc',conn)
df_loincs.head(5)

,loinc_num,component,property,time_aspct,system,scale_typ,method_typ,class,versionlastchanged,chng_type,...,common_test_rank,common_order_rank,common_si_test_rank,hl7_attachment_structure,external_copyright_link,paneltype,askatorderentry,associatedobservations,versionfirstreleased,validhl7attachmentrequest
0,10032-1,R' wave duration.lead V5,Time,Pt,Heart,Qn,EKG,EKG.MEAS,2.48,MIN,...,0,0,0,None,None,None,None,None,1.0i,None
1,1003-3,Indirect antiglobulin test.complement specific...,PrThr,Pt,Ser/Plas,Ord,None,BLDBK,2.56,MIN,...,227,108,227,None,None,None,None,None,1.0,None
2,10033-9,R' wave duration.lead V6,Time,Pt,Heart,Qn,EKG,EKG.MEAS,2.48,MIN,...,0,0,0,None,None,None,None,None,1.0i,None
3,10034-7,S wave amplitude.lead AVF,Elpot,Pt,Heart,Qn,EKG,EKG.MEAS,2.48,MIN,...,0,0,0,None,None,None,None,None,1.0i,None
4,10035-4,S wave amplitude.lead AVL,Elpot,Pt,Heart,Qn,EKG,EKG.MEAS,2.48,MIN,...,0,0,0,None,None,None,None,None,1.0i,None


# Mapping MIMIC-III items to LOINC codes

Clearly, the information held in each LOINC number is useful if we can use that to identify and qualify data in the mimic-iii database. The concept of label mapping between disparate clinical databases is not uncommon, and is in fact a perpetual frustration which drives the need for standardized coding databases such as LOINC. This is not simply for understanding retrospective data, but also for safe and accurate exchange of healthcare information between EMR systems in realtime.In this case, our goal is to map MIMIC-III itemids to LOINC codes, thereby allowing us to leverage the wealth of data living in the LOINC database to help categorize and clean MIMIC-III data. 

Fortunately. Some of this has already been accomplished by some pervious work in the area: **_Abhyankar, Swapna, Dina Demner-Fushman, and Clement J. McDonald. "Standardizing clinical laboratory data for secondary use." Journal of biomedical informatics 45.4 (2012): 642-650. http://www.sciencedirect.com/science/article/pii/S1532046412000652_**

In this paper, they "were able to map 87% of the unique laboratory tests that cover 94% of the total number of laboratory tests results." This was done on the MIMIC-II database, and those mapped codes have now been permamnently added to the MIMIC-III database.

Lets investigate!

### Observations vs. Interventions

First, it is important to acknowledge the limitations of LOINC in the context of this project. We are using 2 different types of data - observational data, and intervention data. LOINC is clearly purposed for the former and NOT the latter. Therefore, we will provide a utility to map as many of our observations as possible to the LOINC definitions, and, when not possible or when the desired data comes from an intervention, we will provide information ourselves on how this should look.

## Observations: LOINC

Our basic process for this:
1. Search for the best LOINC via https://search.loinc.org/
2. Add that to our database

In [404]:
import definitions
import pandas as pd
import utils
import mimic_explorer
import uom
import constants

In [410]:
reload(definitions)
reload(utils)
defs_hdf5_fname = 'config/definitions'
loinc_accesor = definitions.loinc_accessor('postgres','123','loinc')
builder = definitions.builder(defs_hdf5_fname,[loinc_accesor])

Start with **vital signs**. Will be using LOINC panel 34565-2 ("Vital signs, weight and height panel") as a point of reference

https://r.details.loinc.org/LOINC/34565-2.html?sections=Comprehensive

Normal range: https://medlineplus.gov/ency/article/002341.htm


In [ ]:
component = 
units = 
upper_limit =
lower_limit =
variable_type = 
clinical_source =
category_list =
standard_db_name =
standard_db_id =

In [411]:
var_type=constants.variable_type.QUANTITATIVE,
source_type=constants.clinical_source.OBSERVATION


#Heart Rate - 8867-4
builder.add_standard('8867-4','loinc',component='Heart Rate',units='beats/min')

#Systolic Blood Pressure - 8480-6
builder.add_standard('8480-6','loinc',component='Systolic Blood Pressure',units='mmHg')

#Diastolic Blood Pressure - 8462-4
builder.add_standard('8462-4','loinc',component='Diastolic Blood Pressure',units='mmHg')

#Respiratory Rate - 9279-1 
builder.add_standard('9279-1','loinc',component='Respiratory Rate',units='insp/min')

#Temperature - 8310-5 
builder.add_standard('8310-5','loinc',component='Temperature',units='DegC')

builder.add(component='Temperature',units='DegF',
            var_type=constants.variable_type.QUANTITATIVE,
            source_type=constants.clinical_source.OBSERVATION
            )

#O2Sat - 59408-5
builder.add_standard('59408-5','loinc',component='Oxygen Saturation',units='percent')

definitions.py:86: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['component', 'units', 'variable_type', 'clinical_source', 'category_list_id', 'reference_id', 'reference_db']]

  def remove(self,component,units):
definitions.py:79: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['component', 'units', 'variable_type', 'clinical_source', 'category_list_id', 'reference_id', 'reference_db']]

  self.df = pd.concat([self.df,new_row])


In [412]:
utils.open_df(defs_hdf5_fname,'components')

,component,units,variable_type,clinical_source,category_list_id,reference_id,reference_db,component_id
0,Heart rate,beat / min,Qn,observation,None,8867-4,loinc,0
1,Intravascular systolic,mmHg,Qn,observation,None,8480-6,loinc,1
2,Intravascular diastolic,mmHg,Qn,observation,None,8462-4,loinc,2
3,Breaths,br / min,Qn,observation,None,9279-1,loinc,3
4,Body temperature,celsius,Qn,observation,None,8310-5,loinc,4
5,Oxygen saturation,percent,Qn,observation,None,59408-5,loinc,5
6,Body weight,lb,Qn,observation,None,29463-7,loinc,6
7,Body weight,kg,Qn,observation,None,29463-7,loinc,7
8,Fluid output.urine,cubic_centimeter,Qn,observation,None,9187-6,loinc,8
9,Fluid output.urine,ml / h,Qn,observation,None,9188-4,loinc,9


Now **weight**.

In [192]:
#Weight lbs - 29463-7 
builder.add_standard('29463-7','loinc',which_units=0)

#Weight lbs - 29463-7 
builder.add_standard('29463-7','loinc',which_units=1)

utils.open_df(defs_hdf5_fname,'components')

,component,units,variable_type,clinical_source,category_list_id,reference_id,reference_db,component_id
0,Heart rate,beat / min,Qn,observation,None,8867-4,loinc,0
1,Intravascular systolic,mmHg,Qn,observation,None,8480-6,loinc,1
2,Intravascular diastolic,mmHg,Qn,observation,None,8462-4,loinc,2
3,Breaths,br / min,Qn,observation,None,9279-1,loinc,3
4,Body temperature,celsius,Qn,observation,None,8310-5,loinc,4
5,Oxygen saturation,percent,Qn,observation,None,59408-5,loinc,5
6,Body weight,lb,Qn,observation,None,29463-7,loinc,6
7,Body weight,kg,Qn,observation,None,29463-7,loinc,7


**Urine Output**

In [193]:
#UOP cc (cm^3)
builder.add_standard('9187-6','loinc')
#UOP mL/h

builder.add_standard('9188-4','loinc')

#UOP mL/kg/hr
builder.add(component='Urine output',units='ml/kg/hr',
            var_type=constants.variable_type.QUANTITATIVE,
            source_type=constants.clinical_source.OBSERVATION
            )

utils.open_df(defs_hdf5_fname,'components')

,component,units,variable_type,clinical_source,category_list_id,reference_id,reference_db,component_id
0,Heart rate,beat / min,Qn,observation,None,8867-4,loinc,0
1,Intravascular systolic,mmHg,Qn,observation,None,8480-6,loinc,1
2,Intravascular diastolic,mmHg,Qn,observation,None,8462-4,loinc,2
3,Breaths,br / min,Qn,observation,None,9279-1,loinc,3
4,Body temperature,celsius,Qn,observation,None,8310-5,loinc,4
5,Oxygen saturation,percent,Qn,observation,None,59408-5,loinc,5
6,Body weight,lb,Qn,observation,None,29463-7,loinc,6
7,Body weight,kg,Qn,observation,None,29463-7,loinc,7
8,Fluid output.urine,cubic_centimeter,Qn,observation,None,9187-6,loinc,8
9,Fluid output.urine,ml / h,Qn,observation,None,9188-4,loinc,9


**GCS**

In [194]:
#GCS - motor
builder.add_standard('9268-4','loinc')

#GCS - verbal
builder.add_standard('9270-0','loinc')

#GCS - eye opening
builder.add_standard('9267-6','loinc')

utils.open_df(defs_hdf5_fname,'components')

,component,units,variable_type,clinical_source,category_list_id,reference_id,reference_db,component_id
0,Heart rate,beat / min,Qn,observation,None,8867-4,loinc,0
1,Intravascular systolic,mmHg,Qn,observation,None,8480-6,loinc,1
2,Intravascular diastolic,mmHg,Qn,observation,None,8462-4,loinc,2
3,Breaths,br / min,Qn,observation,None,9279-1,loinc,3
4,Body temperature,celsius,Qn,observation,None,8310-5,loinc,4
5,Oxygen saturation,percent,Qn,observation,None,59408-5,loinc,5
6,Body weight,lb,Qn,observation,None,29463-7,loinc,6
7,Body weight,kg,Qn,observation,None,29463-7,loinc,7
8,Fluid output.urine,cubic_centimeter,Qn,observation,None,9187-6,loinc,8
9,Fluid output.urine,ml / h,Qn,observation,None,9188-4,loinc,9


**Lactate**

In [200]:
#Lactate - mmol/L
builder.add_standard('32693-4','loinc')
#Lactate - mg/dL
builder.add_standard('59032-3','loinc')

## Interventions

In [195]:
component = 'Norepinephrine'
units = ['mg','mcg','mcg/min','mcg/kg/min']

for unit in units:
    builder.add(component=component,units=unit,
                var_type=constants.variable_type.QUANTITATIVE,
                source_type=constants.clinical_source.INTERVENTION
                )

In [196]:
component = 'Vasopressin'
units = ['mL','U','U/min','U/hr']

for unit in units:
    builder.add(component=component,units=unit,
                var_type=constants.variable_type.QUANTITATIVE,
                source_type=constants.clinical_source.INTERVENTION
                )

In [197]:
component = 'Normal Saline'
units = ['mL','L','mL/min','mL/hr']

for unit in units:
    builder.add(component=component,units=unit,
                var_type=constants.variable_type.QUANTITATIVE,
                source_type=constants.clinical_source.INTERVENTION
                )

In [198]:
component = 'Lactated Ringers'
units = ['mL','L','mL/min','mL/hr']

for unit in units:
    builder.add(component=component,units=unit,
                var_type=constants.variable_type.QUANTITATIVE,
                source_type=constants.clinical_source.INTERVENTION
                )

In [201]:
utils.open_df(defs_hdf5_fname,'components')

,component,units,variable_type,clinical_source,category_list_id,reference_id,reference_db,component_id
0,Heart rate,beat / min,Qn,observation,None,8867-4,loinc,0
1,Intravascular systolic,mmHg,Qn,observation,None,8480-6,loinc,1
2,Intravascular diastolic,mmHg,Qn,observation,None,8462-4,loinc,2
3,Breaths,br / min,Qn,observation,None,9279-1,loinc,3
4,Body temperature,celsius,Qn,observation,None,8310-5,loinc,4
5,Oxygen saturation,percent,Qn,observation,None,59408-5,loinc,5
6,Body weight,lb,Qn,observation,None,29463-7,loinc,6
7,Body weight,kg,Qn,observation,None,29463-7,loinc,7
8,Fluid output.urine,cubic_centimeter,Qn,observation,None,9187-6,loinc,8
9,Fluid output.urine,ml / h,Qn,observation,None,9188-4,loinc,9


# Finding this data in MIMIC III

In [235]:
import mimic_explorer

In [374]:
reload(mimic_explorer)
reload(definitions)
exp = mimic_explorer.explorer('postgres','123','mimic')
comp_dict = definitions.component_dictionary('config/definitions')
keep_dict = {}

## Lactate

In [312]:
out_df = exp.search(['lactate','lactic acid','mmol/L','mg/dL'])
out_df.head(20)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
50813,LACTATE,NaN,labevents,BLOOD GAS,NaN,100.000000
1531,Lactic Acid,,chartevents,Chemistry,,100.000000
225668,Lactic Acid,Lactic Acid,chartevents,Labs,None,100.000000
220228,Hemoglobin,Hemoglobin,chartevents,Labs,g/dl,92.666667
2773,sjlactate,,chartevents,,,92.000000
225835,Na Phos,Na Phos,inputevents_mv,Medications,mmol,86.666667
225925,Potassium Phosphate,Potassium Phosphate,inputevents_mv,Nutrition - Supplements,mmol,86.666667
227526,Citrate,Citrate,inputevents_mv,Medications,mmol,86.666667
225834,K Phos,K Phos,inputevents_mv,Medications,mmol,86.666667


In [327]:
keep_dict[comp_dict.LACTATE.name] = [1531,50813,225668,818]

## Heart Rate

In [356]:
out_df = exp.search(['heart rate','beats','bpm','pulse'])
out_df.head(20)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
223765,Orthostatic HR sitting,Orthostatic HR sitting,chartevents,Routine Vital Signs,bpm,100.000000
223775,VAD Beat Rate R,VAD Beat Rate R,chartevents,Hemodynamics,bpm,100.000000
1341,PULSE,,chartevents,,,100.000000
224363,VAD Beat Rate L,VAD Beat Rate L,chartevents,Hemodynamics,bpm,100.000000
1725,Pulse,,chartevents,,,100.000000
224751,Temporary Pacemaker Rate,Temp Pacemaker Rate,chartevents,Cardiovascular (Pacer Data),bpm,100.000000
220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,100.000000
211,Heart Rate,,chartevents,,,100.000000
1332,pulse,,chartevents,,,100.000000


In [357]:
keep_dict[comp_dict.HEART_RATE.name] = [211,220045,1341,1725,1332]

## Breaths/Respiratory Rate

In [316]:
out_df = exp.search(['respiratory rate','resp rate','insp/min','breath/min'])
out_df.head(20)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
224161,Resp Alarm - High,Resp Alarm - High,chartevents,Alarms,insp/min,100.000000
226774,RRApacheIIValue,RRApacheIIValue,chartevents,Scores - APACHE II,insp/min,100.000000
220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,100.000000
224688,Respiratory Rate (Set),Respiratory Rate (Set),chartevents,Respiratory,insp/min,100.000000
224689,Respiratory Rate (spontaneous),Respiratory Rate (spontaneous),chartevents,Respiratory,insp/min,100.000000
618,Respiratory Rate,,chartevents,,,100.000000
224690,Respiratory Rate (Total),Respiratory Rate (Total),chartevents,Respiratory,insp/min,100.000000
3603,Resp Rate,,chartevents,,,100.000000
224162,Resp Alarm - Low,Resp Alarm - Low,chartevents,Alarms,insp/min,100.000000


In [317]:
keep_dict[comp_dict.BREATHS] = [220210,3603,618,8113]

## Body Temp

In [318]:
out_df = exp.search(['temperature','temp','celcius','farenheit','degrees','deg'])
out_df.head(20)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
50825,TEMPERATURE,NaN,labevents,BLOOD GAS,NaN,100.000000
223762,Temperature Celsius,Temperature C,chartevents,Routine Vital Signs,?C,94.666667
678,Temperature F,,chartevents,,,94.666667
226170,Head of Bead Measurement,HOB Measurement,chartevents,Treatments,Degree,94.666667
223761,Temperature Fahrenheit,Temperature F,chartevents,Routine Vital Signs,?F,94.666667
676,Temperature C,,chartevents,,,94.666667
591,RLE [Temperature],,chartevents,,,88.000000
597,RUE [Temperature],,chartevents,,,88.000000
224642,Temperature Site,Temp Site,,Routine Vital Signs,,87.333333


In [321]:
keep_dict[comp_dict.BODY_TEMPERATURE.name] = [223761,678,223762,676]

## Blood Pressure

In [322]:
out_df = exp.search(['blood pressure','systolic','diastolic','mmHg'])
out_df.iloc[0:20]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
224314,ABI Brachial BP R (Impella),ABI Brachial BP R (Impella),chartevents,Impella,mmHg,100.0
224645,Orthostatic BPs lying,Orthostatic BPs lying,chartevents,Routine Vital Signs,mmHg,100.0
220235,Arterial CO2 Pressure,PCO2 (Arterial),chartevents,Labs,mmHg,100.0
220224,Arterial O2 pressure,PO2 (Arterial),chartevents,Labs,mmHg,100.0
220181,Non Invasive Blood Pressure mean,NBPm,chartevents,Routine Vital Signs,mmHg,100.0
220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,100.0
220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,100.0
220074,Central Venous Pressure,CVP,chartevents,Hemodynamics,mmHg,100.0
220073,Central Venous Pressure Alarm - Low,CVP Alarm - Low,chartevents,Alarms,mmHg,100.0


In [338]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] = [220179]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] = [220180]

In [339]:
out_df.iloc[20:40]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
226062,Venous CO2 Pressure,PCO2 (Venous),chartevents,Labs,mmHg,100.0
226063,Venous O2 Pressure,PO2 (Venous),chartevents,Labs,mmHg,100.0
224166,Doppler BP,Doppler BP,chartevents,Routine Vital Signs,mmHg,100.0
224167,Manual Blood Pressure Systolic Left,Manual BPs L,chartevents,Routine Vital Signs,mmHg,100.0
225312,ART BP mean,ART BP mean,chartevents,Routine Vital Signs,mmHg,100.0
225310,ART BP Diastolic,ART BP Diastolic,chartevents,Routine Vital Signs,mmHg,100.0
225309,ART BP Systolic,ART BP Systolic,chartevents,Routine Vital Signs,mmHg,100.0
226737,AaDO2ApacheIIValue,AaDO2ApacheIIValue,chartevents,Scores - APACHE II,mmHg,100.0
226096,Orthostatic BPd standing,Orthostatic BPd standing,chartevents,Routine Vital Signs,mmHg,100.0


In [340]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] += [225309,224167]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] += [225310,224643]

In [341]:
out_df.iloc[40:60]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
224150,Filter Pressure,Filter Pressure,chartevents,Dialysis,mmHg,100.0
224151,Effluent Pressure,Effluent Pressure,chartevents,Dialysis,mmHg,100.0
224309,Assisted Systole,SYS - Assisted,chartevents,IABP,mmHg,100.0
224152,Return Pressure,Return Pressure,chartevents,Dialysis,mmHg,100.0
224310,Augmented Diastole,AUG,chartevents,IABP,mmHg,100.0
224311,BAEDP,DIA - Assisted,chartevents,IABP,mmHg,100.0
224315,ABI Brachial BP L,ABI Brachial BP L,chartevents,IABP,mmHg,100.0
224317,ABI Ankle BP R,ABI Ankle BP R,chartevents,IABP,mmHg,100.0
224322,IABP Mean,MAP - Assisted,chartevents,IABP,mmHg,100.0


In [342]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] += [220050]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] += [220051]

In [343]:
out_df.iloc[60:80]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
227023,MAP_ApacheIV,MAP_ApacheIV,chartevents,Scores - APACHE IV (2),mmHg,100.0
223771,PCWP,PCWP,chartevents,Hemodynamics,mmHg,100.0
223768,Intra Cranial Pressure Alarm - Low,ICP Alarm - Low,chartevents,Alarms,mmHg,100.0
228149,ABI Brachial BP L (Impella),ABI Brachial BP L (Impella),chartevents,Impella,mmHg,100.0
227990,Intra Cranial Pressure #2 Alarm - High,IC2 Alarm - High,chartevents,Alarms,mmHg,100.0
226851,RV diastolic pressure(PA Line),RV diastolic pressure(PA Line),chartevents,PA Line Insertion,mmHg,100.0
226853,PA diastolic pressure(PA Line),PA diastolic pressure(PA Line),chartevents,PA Line Insertion,mmHg,100.0
226766,MapApacheIIValue,MapApacheIIValue,chartevents,Scores - APACHE II,mmHg,100.0
227717,Cerebral Perfusion Pressure Alarm - Low,CPP Alarm - Low,chartevents,Alarms,mmHg,100.0


In [344]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] += [227243]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] += [227242]

In [345]:
out_df.iloc[80:100]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
224659,Vacuum Assist,Vacuum Assist,chartevents,Hemodynamics,mmHg,100.000000
224652,Unassisted Systole,SYS - Unassisted,chartevents,IABP,mmHg,100.000000
224654,PAEDP,DIA - Unassisted,chartevents,IABP,mmHg,100.000000
7643,RVSYSTOLIC,,chartevents,,,92.666667
8441,NBP [Diastolic],,chartevents,,,85.666667
8364,ABP [Diastolic],,chartevents,,,85.666667
8448,PAP [Diastolic],,chartevents,,,85.666667
2027,Low pressure,,chartevents,,,84.333333
455,NBP [Systolic],,chartevents,,,84.333333


In [346]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] += [455,6]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] += [8441,8364]

In [347]:
out_df.iloc[100:120]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
8502,BP Cuff [Diastolic],,chartevents,,,77.666667
1404,Bladder pressure,,chartevents,,,77.000000
996,BLADDER PRESSURE,,chartevents,,,77.000000
993,bladder pressure,,chartevents,,,77.000000
1411,Bladder Pressure,,chartevents,,,77.000000
46780,Bladdar pressure,,inputevents_cv,,,77.000000
44862,BLADDER PRESSURE,,inputevents_cv,Free Form Intake,,77.000000
3313,BP Cuff [Systolic],,chartevents,,,76.333333
2347,BLADDER PRESSURE.,,chartevents,,,76.000000


In [348]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] += [3313]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] += [8502,8440]

In [349]:
out_df.iloc[120:140]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
442,Manual BP [Systolic],,chartevents,,,73.000000
6302,low press,,chartevents,,,72.666667
43744,NS bladder pressure,,inputevents_cv,Free Form Intake,,72.333333
7124,Low pressure Alarm,,chartevents,,,72.333333
45682,bladder pressure in,,inputevents_cv,Free Form Intake,,72.333333
2562,INTRA ABD PRESSURE,,chartevents,,,72.333333
8368,Arterial BP [Diastolic],,chartevents,,,72.000000
8503,BP Left Arm [Diastolic],,chartevents,,,72.000000
8504,BP Left Leg [Diastolic],,chartevents,,,72.000000


In [350]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] += [442,3315,51,3317]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] += [8368,8503,8504,8507,8506]

In [351]:
out_df.iloc[140:160]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
2704,ABDOMINAL PRESSURE,,chartevents,,,69.666667
3321,BP Right Arm [Systolic],,chartevents,,,69.000000
3323,BP Right Leg [Systolic],,chartevents,,,69.000000
8555,Arterial BP #2 [Diastolic],,chartevents,,,69.000000
70015,BLOOD,,microbiologyevents,SPECIMEN,,68.666667
6864,Low insp pressure,,chartevents,,,68.666667
51466,BLOOD,NaN,labevents,HEMATOLOGY,NaN,68.666667
3143,low insp pressure,,chartevents,,,68.666667
44560,blood,,inputevents_cv,,,68.666667


In [352]:
keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC] += [3321,3323]
keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC] += [8555]

In [353]:
out_df.loc[keep_dict[comp_dict.INTRAVASCULAR_SYSTOLIC]]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
220179,Non Invasive Blood Pressure systolic,NBPs,chartevents,Routine Vital Signs,mmHg,100.000000
225309,ART BP Systolic,ART BP Systolic,chartevents,Routine Vital Signs,mmHg,100.000000
224167,Manual Blood Pressure Systolic Left,Manual BPs L,chartevents,Routine Vital Signs,mmHg,100.000000
220050,Arterial Blood Pressure systolic,ABPs,chartevents,Routine Vital Signs,mmHg,100.000000
227243,Manual Blood Pressure Systolic Right,Manual BPs R,chartevents,Routine Vital Signs,mmHg,100.000000
455,NBP [Systolic],,chartevents,,,84.333333
6,ABP [Systolic],,chartevents,,,84.333333
3313,BP Cuff [Systolic],,chartevents,,,76.333333
442,Manual BP [Systolic],,chartevents,,,73.000000


In [354]:
out_df.loc[keep_dict[comp_dict.INTRAVASCULAR_DIASTOLIC]]

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
220180,Non Invasive Blood Pressure diastolic,NBPd,chartevents,Routine Vital Signs,mmHg,100.000000
225310,ART BP Diastolic,ART BP Diastolic,chartevents,Routine Vital Signs,mmHg,100.000000
224643,Manual Blood Pressure Diastolic Left,Manual BPd L,chartevents,Routine Vital Signs,mmHg,100.000000
220051,Arterial Blood Pressure diastolic,ABPd,chartevents,Routine Vital Signs,mmHg,100.000000
227242,Manual Blood Pressure Diastolic Right,Manual BPd R,chartevents,Routine Vital Signs,mmHg,100.000000
8441,NBP [Diastolic],,chartevents,,,85.666667
8364,ABP [Diastolic],,chartevents,,,85.666667
8502,BP Cuff [Diastolic],,chartevents,,,77.666667
8440,Manual BP [Diastolic],,chartevents,,,74.666667


## O2 Sat

In [359]:
out_df = exp.search(['oxygen saturation','O2sat','pulse oximetry','%','spo2'])
out_df.head(40)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
50817,OXYGEN SATURATION,NaN,labevents,BLOOD GAS,NaN,100.000000
646,SpO2,,chartevents,,,100.000000
220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,100.000000
228232,PAR-Oxygen saturation,PAR-Oxygen saturation,,Routine Vital Signs,,92.666667
3785,PO2,,chartevents,ABG's,,90.666667
50821,PO2,NaN,labevents,BLOOD GAS,NaN,90.666667
3837,pO2,,chartevents,ABG'S,,90.666667
6719,SpO2-L,,chartevents,,,86.666667
2955,JVO2 SAT,,chartevents,,,78.000000


In [360]:
keep_dict[comp_dict.OXYGEN_SATURATION.name] = [646,220277,228232]

## Urine Output

In [378]:
out_df = exp.search(['urine out','void','foley'])
out_df.head(20)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
226560,Void,Void,outputevents,Output,mL,110.000000
43332,void,,inputevents_cv,,,110.000000
7672,Foley,,chartevents,,,110.000000
3686,Void,,chartevents,,,110.000000
45967,foley,,inputevents_cv,,,110.000000
43053,URINE OUT,,outputevents,,,110.000000
226559,Foley,Foley,outputevents,Output,mL,110.000000
44103,ER urine out,,outputevents,,,100.666667
44834,er urine out,,outputevents,,,100.666667


In [379]:
out_df.head(40)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
226560,Void,Void,outputevents,Output,mL,110.000000
43332,void,,inputevents_cv,,,110.000000
7672,Foley,,chartevents,,,110.000000
3686,Void,,chartevents,,,110.000000
45967,foley,,inputevents_cv,,,110.000000
43053,URINE OUT,,outputevents,,,110.000000
226559,Foley,Foley,outputevents,Output,mL,110.000000
44103,ER urine out,,outputevents,,,100.666667
44834,er urine out,,outputevents,,,100.666667


In [380]:
out_df = out_df[out_df.linksto.isin(['outputevents','chartevents'])]

In [381]:
out_df.head(40)

,label,abbreviation,linksto,category,unitname,score
itemid,,,,,,
226560,Void,Void,outputevents,Output,mL,110.000000
7672,Foley,,chartevents,,,110.000000
3686,Void,,chartevents,,,110.000000
43053,URINE OUT,,outputevents,,,110.000000
226559,Foley,Foley,outputevents,Output,mL,110.000000
44103,ER urine out,,outputevents,,,100.666667
44834,er urine out,,outputevents,,,100.666667
227519,Urine output_ApacheIV,Urine output,chartevents,Scores - APACHE IV (2),None,100.666667
44706,urine output,,outputevents,,,100.666667


In [390]:
import pint

In [391]:
ureg = pint.UnitRegistry()

In [397]:
ureg.parse_units('degF')

<Unit('degF')>